In [ ]:
import time, socket, sys, pickle
import random
from sympy import *

def coprime(a, b):
    while b != 0:
        a, b = b, a % b
    return a
    
    
def extended_gcd(aa, bb):
    lastremainder, remainder = abs(aa), abs(bb)
    x, lastx, y, lasty = 0, 1, 1, 0
    while remainder:
        lastremainder,(quotient,remainder)=remainder,divmod(lastremainder,remainder)
        x, lastx = lastx - quotient*x, x
        y, lasty = lasty - quotient*y, y
    return lastremainder,lastx*(-1 if aa < 0 else 1),lasty*(-1 if bb < 0 else 1)

#Euclid's extended algorithm for finding the multiplicative inverse of two numbers    
def modinv(a, m):
	g, x, y = extended_gcd(a, m)
	if g != 1:
		raise Exception('Modular inverse does not exist')
	return x % m    


def generate_keypair():
    while(True):
        p=random.randint(100, 200)
        if(isprime(p)):
            break;
    #picking random primes such that p!=q 
    while(True):
        while(True):
            q=random.randint(100, 200)
            if(isprime(q)):
                break;
        if(p!=q):
            break;
    n = p * q
    phi = (p-1) * (q-1)
    #Choose an integer e such that e and phi(n) are coprime
  
    while(True):
        e = random.randrange(1, phi)
        g = coprime(e, phi)
        if(g==1):
            break;
    #Use Extended Euclid's Algorithm to generate the private key
    d = modinv(e, phi)
    #Return public and private keypair
    return ((e, n), (d, n))


def encrypt(privatek, plaintext):
    key, n = privatek
    #Convert each letter in the plaintext to numbers based on the character
    numberRepr = [ord(char) for char in plaintext]
    print("Number representation of message before signature generation: ",\
          numberRepr)
    cipher = [pow(ord(char),key,n) for char in plaintext]
    print("Number representation of signature: ", cipher)
    return cipher
def decrypt(publick, ciphertext):
    key, n = publick
    #Generate the plaintext based on the ciphertext and key
    numberRepr = [pow(char, key, n) for char in ciphertext]
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    print("Decrypted signature's number representation is: ", numberRepr)
    return ''.join(plain)
#creating a socket         
socket_server = socket.socket()
server_host = socket.gethostname()
ip = socket.gethostbyname(server_host)
sport = 5000
 
print('This is your IP address: ',ip)
server_host = input('Enter friend\'s IP address:')
name = input('Enter your name: ')
 
 
socket_server.connect((server_host, sport))
 
socket_server.send(name.encode())
server_name = socket_server.recv(1024)
server_name = server_name.decode()
 
print(server_name,' has joined...')

e=int((socket_server.recv(1024)).decode())
n=int((socket_server.recv(1024)).decode())
public1=(e,n)
print("recived the public key of",server_name," :("+str(e)+","+str(n)+")")
print("Generating public and private keypairs of",name,"...")
public, private = generate_keypair()
e1,n1=public
d1,n1=private
print("Public key of",name," is :("+str(e1)+","+str(n1)+ ") "\
      "Private key of",name," is :("+str(d1)+","+str(n1)+")")
print("Sending public key of",name,"to",server_name,"...")
socket_server.send(str(e1).encode())
socket_server.send(str(n1).encode())
print("sent public key.")
while True:
    message = (socket_server.recv(1024)).decode()
    sign = pickle.loads((socket_server.recv(1024)))
    print(server_name, " message:", message)
    print(server_name, " signature:", sign)
    print("verifing the signature using",server_name,\
          "'s public key :(" +str(e1)+","+str(n1)+")")
    m1 = decrypt(public1, sign)
    if(m1==message):
        print("Verified signature,Message is from ",server_name)
    message = input("Me : ")
    print("generating signature with",name,"'s private key :", private ," . . .")
    sign_msg = encrypt(private, message)
    socket_server.send(message.encode())
    socket_server.send(pickle.dumps(sign_msg))
    print("message,signature sent.")
      

This is your IP address:  192.168.1.34
Enter friend's IP address:192.168.1.34
Enter your name: Bob
Alice  has joined...
recived the public key of Alice  :(11573,19291)
Generating public and private keypairs of Bob ...
Public key of Bob  is :(5351,17399) Private key of Bob  is :(13655,17399)
Sending public key of Bob to Alice ...
sent public key.
Alice  message: HI,this is alice
Alice  signature: [5832, 175, 4412, 5829, 4436, 18276, 15156, 17535, 18276, 15156, 17535, 16872, 10539, 18276, 13075, 15049]
verifing the signature using Alice 's public key :(5351,17399)
Decrypted signature's number representation is:  [72, 73, 44, 116, 104, 105, 115, 32, 105, 115, 32, 97, 108, 105, 99, 101]
Verified signature,Message is from  Alice
Me : hi, i am bob
generating signature with Bob 's private key : (13655, 17399)  . . .
Number representation of message before signature generation:  [104, 105, 44, 32, 105, 32, 97, 109, 32, 98, 111, 98]
Number representation of signature:  [6518, 7390, 2534, 10557,